In [28]:
def install_packages(s3_bucket, s3_key, new_packages):
    # Create a session using your credentials
    s3 = boto3.client('s3')

    # The directory name is assumed to be the same as the s3_key without the extension
    dir_name = '/tmp/' + os.path.splitext(s3_key)[0]

    # Check if the directory already exists
    if not os.path.isdir(dir_name):
        # If the directory does not exist, download and unzip the file from S3
        try:
            with open('/tmp/temp.zip', 'wb') as f:
                s3.download_fileobj(s3_bucket, s3_key, f)
            with zipfile.ZipFile('/tmp/temp.zip', 'r') as zip_ref:
                zip_ref.extractall(dir_name)
            os.remove('/tmp/temp.zip')
        except NoCredentialsError:
            print("No AWS credentials found.")
            return

    # Add the directory to sys.path
    sys.path.append(dir_name)

    # Import the packages and print their versions
    for package in new_packages:
        try:
            module = importlib.import_module(package)
            print(f"{package} version: {module.__version__}")
        except ImportError:
            print(f"Failed to import {package}. Please make sure it is installed and available.")

In [ ]:
import os
import zipfile
import subprocess
import sys

def download_package_and_dependencies(package):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'download', package])
    except Exception as e:
        print(f"An error occurred while downloading {package}: {e}")

def create_zip_folder():
    # Name of the zip file
    zip_name = 'packages.zip'
    
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        # Iterate over all the files in the current directory
        for file in os.listdir():
            # Add file to the zip
            zipf.write(file)
    
    print(f"Zip folder '{zip_name}' created successfully.")

# Download scipy and its dependencies
download_package_and_dependencies('scipy')

# Create zip folder for downloaded packages
create_zip_folder()


In [30]:
import os
import zipfile
import subprocess
import sys

def download_package_and_dependencies(package, directory):
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Change the current working directory
    os.chdir(directory)
    
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'download', package])
    except Exception as e:
        print(f"An error occurred while downloading {package}: {e}")
    
    # Change back to the original directory
    os.chdir('..')

def create_zip_folder(directory):
    # Name of the zip file
    zip_name = 'packages.zip'
    
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        # Iterate over all the files in the specified directory
        for file in os.listdir(directory):
            # Add file to the zip
            zipf.write(os.path.join(directory, file))
    
    print(f"Zip folder '{zip_name}' created successfully.")

# Directory to download packages into
directory = 'scipy_packages'

# Download scipy and its dependencies
download_package_and_dependencies('scipy', directory)

# Create zip folder for downloaded packages
create_zip_folder(directory)


Zip folder 'packages.zip' created successfully.


In [33]:
import os
import zipfile
import subprocess
import sys

def download_package_and_dependencies(package, directory):
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Change the current working directory
    os.chdir(directory)
    
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'download', package])
    except Exception as e:
        print(f"An error occurred while downloading {package}: {e}")
    
    # Change back to the original directory
    os.chdir('..')

def create_zip_folder(directory):
    # Name of the zip file
    zip_name = f'{directory}.zip'
    
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        # Iterate over all the files in the specified directory
        for file in os.listdir(directory):
            # Add file to the zip
            zipf.write(os.path.join(directory, file))
    
    print(f"Zip folder '{zip_name}' created successfully.")

# List of packages to download
packages = ['scipy', 'sclearn', 'matplotlib']

# Download each package and its dependencies, then create a zip folder
for package in packages:
    directory = f'{package}_packages'
    download_package_and_dependencies(package, directory)
    create_zip_folder(directory)


Zip folder 'scipy_packages.zip' created successfully.
Zip folder 'scikit-learn_packages.zip' created successfully.
Zip folder 'matplotlib_packages.zip' created successfully.


In [34]:
import os
import zipfile
import subprocess

def download_and_zip_sklearn_utils():
    # Create a new directory for the package
    os.makedirs('python', exist_ok=True)

    # Use pip to download the package
    subprocess.check_call(['pip', 'install', '--target=./python', 'sklearn'])

    # Remove all other sklearn modules except utils
    sklearn_dir = './python/sklearn'
    for filename in os.listdir(sklearn_dir):
        if filename != 'utils':
            file_path = os.path.join(sklearn_dir, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

    # Create a zip file
    with zipfile.ZipFile('sklearn_utils.zip', 'w') as myzip:
        for root, dirs, files in os.walk('python'):
            for file in files:
                myzip.write(os.path.join(root, file))

download_and_zip_sklearn_utils()


CalledProcessError: Command '['pip', 'install', '--target=./python', 'sklearn']' returned non-zero exit status 1.

In [38]:
    from io import StringIO
    import  pandas as pd
    s3 = boto3.client('s3')
    bucket_name = 'useast1-dlx-dev-ddm-datascience'
    object_key = 'AP_folder/dataset/profiling.csv'
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    data = response['Body'].read().decode('utf-8')
    type(data)
    
    
    
    
    data = StringIO(data)
    df = pd.read_csv(data, sep=",")

ParserError: Error tokenizing data. C error: Expected 2 fields in line 28, saw 41


In [43]:
import os
import zipfile
import subprocess

def download_and_zip_package(package_name):
    # Download the package
    subprocess.check_call(["pip", "download", package_name, "--no-deps", "-d", "./packages"])

    # Get the list of downloaded files
    files = os.listdir("./packages")
    print(files)
    # Create a ZipFile object
    with zipfile.ZipFile(f"{package_name}.zip", 'w') as zipf:
        # Add files to the zip file
        for file in files:
            zipf.write(os.path.join("./packages", file))

# Call the function
download_and_zip_package("scipy")


['scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl']


In [42]:
files

NameError: name 'files' is not defined

In [56]:
import os
import shutil
import subprocess

# Create a new directory structure
os.makedirs('python/lib/python3.7/site-packages', exist_ok=True)

# Install packages from requirements.txt into the new directory
subprocess.check_call(['pip', 'install', '-r', 'requirements.txt', '--target', './python/lib/python3.7/site-packages'])

# Zip the directory
shutil.make_archive('package', 'zip', 'python')



'/root/ProfilingTool/package.zip'

'/root/ProfilingTool'